In [1]:
# import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np
import warnings
import sys
import os
warnings.filterwarnings("ignore")
sys.stderr = open(os.devnull, 'w')

# local crs
local_crs = 32636

In [2]:
from popframe.method.popuation_frame import PopulationFrame
from popframe.models.region import Region
# region_model = Region.from_pickle('C:\Code\PopFrame\Ленинградская область.pickle')
region_model = Region.from_pickle('data/Ленинградская область.pickle')

In [3]:
frame_method = PopulationFrame(region=region_model)
G = frame_method.build_network_frame()
num_edges = G.number_of_edges()
print("Количество рёбер в графе:", num_edges)


Количество рёбер в графе: 2929


In [4]:
output = frame_method.build_circle_frame(output_type='gdf') 
output

,id,name,population,level,geometry
0,0,Болото,10,Малое сельское поселение,"POLYGON ((543177.545 6580637.216, 543177.376 6..."
1,1,Большой Остров,68,Малое сельское поселение,"POLYGON ((544624.730 6593227.683, 544624.288 6..."
2,2,Бор,1734,Большое сельское поселение,"POLYGON ((545358.957 6593052.346, 545356.725 6..."
3,3,Бороватое,10,Малое сельское поселение,"POLYGON ((544028.318 6589581.591, 544028.148 6..."
4,4,Бочево,10,Малое сельское поселение,"POLYGON ((538575.316 6576793.156, 538575.147 6..."
...,...,...,...,...,...
2925,2925,Апраксин Бор,313,Среднее сельское поселение,"POLYGON ((399256.679 6560340.947, 399255.730 6..."
2926,2926,Александровка,313,Среднее сельское поселение,"POLYGON ((404164.108 6562086.557, 404163.159 6..."
2927,2927,Большая Горка,313,Среднее сельское поселение,"POLYGON ((413313.678 6573792.753, 413312.730 6..."
2928,2928,Дроздово,5,Малое сельское поселение,"POLYGON ((412482.916 6574810.780, 412482.796 6..."


In [6]:
from popframe.method.aglomeration import AgglomerationBuilder
# Создаем экземпляр класса и запускаем процесс создания агломераций

agglomeration_builder = AgglomerationBuilder(region=region_model)
final_agglomerations = agglomeration_builder.get_agglomerations()
final_agglomerations


,type,core_cities,population,geometry
0,Polycentric,"Никольское, Коммунар, Бугры, Гатчина, Кировск,...",1212757,"POLYGON ((306826.489 6588382.825, 306671.134 6..."
1,Polycentric,"Светогорск, Выборг",156026,"POLYGON ((259598.016 6699075.723, 259443.936 6..."
2,Polycentric,"Бокситогорск, Тихвин, Пикалево",109296,"POLYGON ((507218.130 6590192.614, 507013.937 6..."
3,Polycentric,"Кириши, Волхов",109648,"POLYGON ((418794.336 6582509.234, 418736.220 6..."
4,Polycentric,"Сланцы, Кингисепп",117386,"POLYGON ((238644.633 6578672.709, 238983.404 6..."
5,Polycentric,"Лодейное Поле, Подпорожье",45399,"POLYGON ((510785.306 6710188.935, 510607.006 6..."
6,Monocentric,Приозерск,34262,"POLYGON ((337076.979 6743413.735, 336818.646 6..."


In [7]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from popframe.models.region import Region
from shapely.geometry import shape


geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

# Преобразуем словарь в геометрию
geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territories_gdf = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

evaluation = TerritoryEvaluation(region=region_model)
# Пример вызова функции
results = evaluation.population_criterion(territories_gdf=territories_gdf)
results

[{'project': None,
  'average_population_density': 17.2,
  'total_population': 39739,
  'score': 4.0,
  'interpretation': 'Территория имеет хорошие показатели численности и плотности населения, что способствует ее активному развитию.'}]

In [8]:
from popframe.method.territory_evaluation import TerritoryEvaluation
from shapely.geometry import shape


geojson_data = {
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [29.982879431084967, 59.363554752593245],
                [29.982879431084967, 59.322083801173534],
                [30.109075699649765, 59.322083801173534],
                [30.109075699649765, 59.363554752593245],
                [29.982879431084967, 59.363554752593245]
            ]
        ]
    },
    "properties": {},
    "type": "Feature"
}

geom = shape(geojson_data['geometry'])

# Создаем GeoDataFrame
territories_gdf = gpd.GeoDataFrame([{'geometry': geom}], crs='EPSG:4326')

result = evaluation.evaluate_territory_location(territories_gdf=territories_gdf)
result

[{'territory': 'None',
  'score': 5,
  'interpretation': 'Территория находится внутри или непосредственной близости населенного пункта уровня Малый город',
  'closest_settlement': 'Сиверский',
  'closest_settlement1': None,
  'closest_settlement2': None}]